# Analyst Rating Data Script 
''' This script is designed to take multiple sources to gather, concat, and clean the data set
    to look at 4 main categories that have influence on stock price action (i.e., Technicals, Fundamentals,
    Macro, and News Sentiment.'''

## Part 1: Read, Open, and Clean Fundamental Data

In [26]:
# Import dependencies from alpaca trade and transformers
import pandas as pd
from pathlib import Path
import yfinance as yf
from sklearn.impute import SimpleImputer
import numpy as np

In [27]:
# Set columns and rows as max to be able to better view Data for irregularities/NaN values
pd.set_option('display.max_columns', None)
pd.set_option('max_rows', None)

In [28]:
# Open Cashflow Sheet for AAPL
aapl_cashflow_df = pd.read_csv(
    Path("AAPL_quarterly_cash-flow.csv"),
    index_col = 'name',
    parse_dates=True,
infer_datetime_format=True).T

# View aapl_cashflow_df
aapl_cashflow_df.head()

name,OperatingCashFlow,\tCashFlowFromContinuingOperatingActivities,\t\tNetIncomeFromContinuingOperations,\t\tOperatingGainsLosses,\t\t\tGainLossOnSaleOfBusiness,\t\t\tGainLossOnSaleOfPPE,\t\t\tGainLossOnInvestmentSecurities,\t\tDepreciationAmortizationDepletion,\t\t\tDepreciationAndAmortization,\t\tDeferredTax,\t\t\tDeferredIncomeTax,\t\tUnrealizedGainLossOnInvestmentSecurities,\t\tStockBasedCompensation,\t\tExcessTaxBenefitFromStockBasedCompensation,\t\tOtherNonCashItems,\t\tChangeInWorkingCapital,\t\t\tChangeInReceivables,\t\t\t\tChangesInAccountReceivables,\t\t\tChangeInInventory,\t\t\tChangeInPrepaidAssets,\t\t\tChangeInPayablesAndAccruedExpense,\t\t\t\tChangeInPayable,\t\t\t\t\tChangeInTaxPayable,\t\t\t\t\t\tChangeInIncomeTaxPayable,\t\t\t\t\tChangeInAccountPayable,\t\t\t\tChangeInAccruedExpense,\t\t\tChangeInOtherCurrentAssets,\t\t\tChangeInOtherCurrentLiabilities,\t\t\tChangeInOtherWorkingCapital,InvestingCashFlow,\tCashFlowFromContinuingInvestingActivities,\t\tNetPPEPurchaseAndSale,\t\t\tPurchaseOfPPE,\t\t\tSaleOfPPE,\t\tNetIntangiblesPurchaseAndSale,\t\t\tPurchaseOfIntangibles,\t\tNetBusinessPurchaseAndSale,\t\t\tPurchaseOfBusiness,\t\t\tSaleOfBusiness,\t\tNetInvestmentPurchaseAndSale,\t\t\tPurchaseOfInvestment,\t\t\tSaleOfInvestment,\t\tNetOtherInvestingChanges,FinancingCashFlow,\tCashFlowFromContinuingFinancingActivities,\t\tNetIssuancePaymentsOfDebt,\t\t\tNetLongTermDebtIssuance,\t\t\t\tLongTermDebtIssuance,\t\t\t\tLongTermDebtPayments,\t\t\tNetShortTermDebtIssuance,\t\t\t\tShortTermDebtIssuance,\t\t\t\tShortTermDebtPayments,\t\tNetCommonStockIssuance,\t\t\tCommonStockIssuance,\t\t\tCommonStockPayments,\t\tCashDividendsPaid,\t\t\tCommonStockDividendPaid,\t\tProceedsFromStockOptionExercised,\t\tNetOtherFinancingCharges,EndCashPosition,\tChangesInCash,\tBeginningCashPosition,\tOtherCashAdjustmentOutsideChangeinCash,IncomeTaxPaidSupplementalData,InterestPaidSupplementalData,CapitalExpenditure,IssuanceOfCapitalStock,IssuanceOfDebt,RepaymentOfDebt,RepurchaseOfCapitalStock,FreeCashFlow
ttm,"109,584,000,000","109,584,000,000","94,321,000,000",NaN,NaN,NaN,NaN,"11,484,000,000","11,484,000,000",NaN,NaN,NaN,"10,112,000,000",NaN,"-2,689,000,000","-4,539,000,000","8,853,000,000","2,231,000,000","-2,129,000,000",NaN,"-9,566,000,000","-9,566,000,000",NaN,NaN,"-9,566,000,000",NaN,"-7,049,000,000","4,874,000,000","147,000,000","3,891,000,000","3,891,000,000","-12,094,000,000","-12,094,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"17,583,000,000","-26,133,000,000","43,716,000,000","-1,292,000,000","-115,526,000,000","-115,526,000,000","-8,983,000,000","-3,979,000,000",NaN,"-9,444,000,000","-5,004,000,000",NaN,NaN,"-85,362,000,000",NaN,"-85,362,000,000","-14,932,000,000",NaN,NaN,"-6,249,000,000","27,129,000,000","-2,051,000,000","29,180,000,000",NaN,"15,166,000,000","3,332,000,000","-12,094,000,000",NaN,NaN,"-9,444,000,000","-85,362,000,000","97,490,000,000"
03/31/2023,"28,560,000,000","28,560,000,000","24,160,000,000",NaN,NaN,NaN,NaN,"2,898,000,000","2,898,000,000",NaN,NaN,NaN,"2,686,000,000",NaN,"-1,415,000,000","231,000,000","17,786,000,000","5,321,000,000","-741,000,000",NaN,"-14,689,000,000","-14,689,000,000",NaN,NaN,"-14,689,000,000",NaN,"7,000,000","-2,001,000,000",NaN,"2,319,000,000","2,319,000,000","-2,916,000,000","-2,916,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"5,341,000,000","-6,044,000,000","11,385,000,000","-106,000,000","-25,724,000,000","-25,724,000,000","-1,996,000,000","-2,250,000,000",NaN,"-2,250,000,000","254,000,000",NaN,NaN,"-19,594,000,000",NaN,"-19,594,000,000","-3,650,000,000","-3,650,000,000",NaN,"-484,000,000","27,129,000,000","5,155,000,000","21,974,000,000",NaN,"4,066,000,000","1,170,000,000","-2,916,000,000",NaN,NaN,"5,964,000,000","-19,594,000,000","25,644,000,000"
12/31/2022,"34,005,000,000","34,005,000,000","29,998,000,000",NaN,NaN,NaN,NaN,"2,916,000,000","2,916,000,000",NaN,NaN,NaN,"2,905,000,000",NaN,"-317,000,000","-1,497,000,000","6,595,000,000","4,275,000,000","-1,807,000,000",NaN,"-6,075,000,000","-6,075,000,000",

In [29]:
# Drop 'ttm' from the row 
aapl_cashflow_df = aapl_cashflow_df.drop(labels=('ttm'))

# Check aapl_cashflow_df
aapl_cashflow_df.head()

name,OperatingCashFlow,\tCashFlowFromContinuingOperatingActivities,\t\tNetIncomeFromContinuingOperations,\t\tOperatingGainsLosses,\t\t\tGainLossOnSaleOfBusiness,\t\t\tGainLossOnSaleOfPPE,\t\t\tGainLossOnInvestmentSecurities,\t\tDepreciationAmortizationDepletion,\t\t\tDepreciationAndAmortization,\t\tDeferredTax,\t\t\tDeferredIncomeTax,\t\tUnrealizedGainLossOnInvestmentSecurities,\t\tStockBasedCompensation,\t\tExcessTaxBenefitFromStockBasedCompensation,\t\tOtherNonCashItems,\t\tChangeInWorkingCapital,\t\t\tChangeInReceivables,\t\t\t\tChangesInAccountReceivables,\t\t\tChangeInInventory,\t\t\tChangeInPrepaidAssets,\t\t\tChangeInPayablesAndAccruedExpense,\t\t\t\tChangeInPayable,\t\t\t\t\tChangeInTaxPayable,\t\t\t\t\t\tChangeInIncomeTaxPayable,\t\t\t\t\tChangeInAccountPayable,\t\t\t\tChangeInAccruedExpense,\t\t\tChangeInOtherCurrentAssets,\t\t\tChangeInOtherCurrentLiabilities,\t\t\tChangeInOtherWorkingCapital,InvestingCashFlow,\tCashFlowFromContinuingInvestingActivities,\t\tNetPPEPurchaseAndSale,\t\t\tPurchaseOfPPE,\t\t\tSaleOfPPE,\t\tNetIntangiblesPurchaseAndSale,\t\t\tPurchaseOfIntangibles,\t\tNetBusinessPurchaseAndSale,\t\t\tPurchaseOfBusiness,\t\t\tSaleOfBusiness,\t\tNetInvestmentPurchaseAndSale,\t\t\tPurchaseOfInvestment,\t\t\tSaleOfInvestment,\t\tNetOtherInvestingChanges,FinancingCashFlow,\tCashFlowFromContinuingFinancingActivities,\t\tNetIssuancePaymentsOfDebt,\t\t\tNetLongTermDebtIssuance,\t\t\t\tLongTermDebtIssuance,\t\t\t\tLongTermDebtPayments,\t\t\tNetShortTermDebtIssuance,\t\t\t\tShortTermDebtIssuance,\t\t\t\tShortTermDebtPayments,\t\tNetCommonStockIssuance,\t\t\tCommonStockIssuance,\t\t\tCommonStockPayments,\t\tCashDividendsPaid,\t\t\tCommonStockDividendPaid,\t\tProceedsFromStockOptionExercised,\t\tNetOtherFinancingCharges,EndCashPosition,\tChangesInCash,\tBeginningCashPosition,\tOtherCashAdjustmentOutsideChangeinCash,IncomeTaxPaidSupplementalData,InterestPaidSupplementalData,CapitalExpenditure,IssuanceOfCapitalStock,IssuanceOfDebt,RepaymentOfDebt,RepurchaseOfCapitalStock,FreeCashFlow
03/31/2023,"28,560,000,000","28,560,000,000","24,160,000,000",NaN,NaN,NaN,NaN,"2,898,000,000","2,898,000,000",NaN,NaN,NaN,"2,686,000,000",NaN,"-1,415,000,000","231,000,000","17,786,000,000","5,321,000,000","-741,000,000",NaN,"-14,689,000,000","-14,689,000,000",NaN,NaN,"-14,689,000,000",NaN,"7,000,000","-2,001,000,000",NaN,"2,319,000,000","2,319,000,000","-2,916,000,000","-2,916,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"5,341,000,000","-6,044,000,000","11,385,000,000","-106,000,000","-25,724,000,000","-25,724,000,000","-1,996,000,000","-2,250,000,000",NaN,"-2,250,000,000","254,000,000",NaN,NaN,"-19,594,000,000",NaN,"-19,594,000,000","-3,650,000,000","-3,650,000,000",NaN,"-484,000,000","27,129,000,000","5,155,000,000","21,974,000,000",NaN,"4,066,000,000","1,170,000,000","-2,916,000,000",NaN,NaN,"5,964,000,000","-19,594,000,000","25,644,000,000"
12/31/2022,"34,005,000,000","34,005,000,000","29,998,000,000",NaN,NaN,NaN,NaN,"2,916,000,000","2,916,000,000",NaN,NaN,NaN,"2,905,000,000",NaN,"-317,000,000","-1,497,000,000","6,595,000,000","4,275,000,000","-1,807,000,000",NaN,"-6,075,000,000","-6,075,000,000",NaN,NaN,"-6,075,000,000",NaN,"-4,099,000,000","3,758,000,000","131,000,000","-1,445,000,000","-1,445,000,000","-3,787,000,000","-3,787,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"2,483,000,000","-5,153,000,000","7,636,000,000","-141,000,000","-35,563,000,000","-35,563,000,000","-9,615,000,000","-1,401,000,000",NaN,"-1,401,000,000","-8,214,000,000",NaN,"-8,214,000,000","-19,475,000,000",NaN,"-19,475,000,000","-3,768,000,000","-3,768,000,000",NaN,"-2,705,000,000","21,974,000,000","-3,003,000,000","24,977,000,000",NaN,"828,000,000","703,000,000","-3,787,000,000",NaN,NaN,"-9,615,000,000","-19,475,000,000","30,218,000,000"
09/30/2022,"24,127,000,000","24,127,000,000","20,721,000,000",NaN,NaN,NaN,NaN,"2,865,000,000","2,865,000,000","-1,861,000,000","-1,861,000,000",NaN,"2,278,000,000",NaN,"172,000,000","-48,000,000","-18,693,000,000","-6,384,000,000","435,000,000",NaN,

In [30]:
# Create a new column 'Date' that takes in the datetime format 
aapl_cashflow_df['Date'] = pd.to_datetime(aapl_cashflow_df.index).strftime('%Y-%m-%d')

# View aapl_cashflow_df to make sure that 'Date' has been created as a new column
aapl_cashflow_df.head()

name,OperatingCashFlow,\tCashFlowFromContinuingOperatingActivities,\t\tNetIncomeFromContinuingOperations,\t\tOperatingGainsLosses,\t\t\tGainLossOnSaleOfBusiness,\t\t\tGainLossOnSaleOfPPE,\t\t\tGainLossOnInvestmentSecurities,\t\tDepreciationAmortizationDepletion,\t\t\tDepreciationAndAmortization,\t\tDeferredTax,\t\t\tDeferredIncomeTax,\t\tUnrealizedGainLossOnInvestmentSecurities,\t\tStockBasedCompensation,\t\tExcessTaxBenefitFromStockBasedCompensation,\t\tOtherNonCashItems,\t\tChangeInWorkingCapital,\t\t\tChangeInReceivables,\t\t\t\tChangesInAccountReceivables,\t\t\tChangeInInventory,\t\t\tChangeInPrepaidAssets,\t\t\tChangeInPayablesAndAccruedExpense,\t\t\t\tChangeInPayable,\t\t\t\t\tChangeInTaxPayable,\t\t\t\t\t\tChangeInIncomeTaxPayable,\t\t\t\t\tChangeInAccountPayable,\t\t\t\tChangeInAccruedExpense,\t\t\tChangeInOtherCurrentAssets,\t\t\tChangeInOtherCurrentLiabilities,\t\t\tChangeInOtherWorkingCapital,InvestingCashFlow,\tCashFlowFromContinuingInvestingActivities,\t\tNetPPEPurchaseAndSale,\t\t\tPurchaseOfPPE,\t\t\tSaleOfPPE,\t\tNetIntangiblesPurchaseAndSale,\t\t\tPurchaseOfIntangibles,\t\tNetBusinessPurchaseAndSale,\t\t\tPurchaseOfBusiness,\t\t\tSaleOfBusiness,\t\tNetInvestmentPurchaseAndSale,\t\t\tPurchaseOfInvestment,\t\t\tSaleOfInvestment,\t\tNetOtherInvestingChanges,FinancingCashFlow,\tCashFlowFromContinuingFinancingActivities,\t\tNetIssuancePaymentsOfDebt,\t\t\tNetLongTermDebtIssuance,\t\t\t\tLongTermDebtIssuance,\t\t\t\tLongTermDebtPayments,\t\t\tNetShortTermDebtIssuance,\t\t\t\tShortTermDebtIssuance,\t\t\t\tShortTermDebtPayments,\t\tNetCommonStockIssuance,\t\t\tCommonStockIssuance,\t\t\tCommonStockPayments,\t\tCashDividendsPaid,\t\t\tCommonStockDividendPaid,\t\tProceedsFromStockOptionExercised,\t\tNetOtherFinancingCharges,EndCashPosition,\tChangesInCash,\tBeginningCashPosition,\tOtherCashAdjustmentOutsideChangeinCash,IncomeTaxPaidSupplementalData,InterestPaidSupplementalData,CapitalExpenditure,IssuanceOfCapitalStock,IssuanceOfDebt,RepaymentOfDebt,RepurchaseOfCapitalStock,FreeCashFlow,Date
03/31/2023,"28,560,000,000","28,560,000,000","24,160,000,000",NaN,NaN,NaN,NaN,"2,898,000,000","2,898,000,000",NaN,NaN,NaN,"2,686,000,000",NaN,"-1,415,000,000","231,000,000","17,786,000,000","5,321,000,000","-741,000,000",NaN,"-14,689,000,000","-14,689,000,000",NaN,NaN,"-14,689,000,000",NaN,"7,000,000","-2,001,000,000",NaN,"2,319,000,000","2,319,000,000","-2,916,000,000","-2,916,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"5,341,000,000","-6,044,000,000","11,385,000,000","-106,000,000","-25,724,000,000","-25,724,000,000","-1,996,000,000","-2,250,000,000",NaN,"-2,250,000,000","254,000,000",NaN,NaN,"-19,594,000,000",NaN,"-19,594,000,000","-3,650,000,000","-3,650,000,000",NaN,"-484,000,000","27,129,000,000","5,155,000,000","21,974,000,000",NaN,"4,066,000,000","1,170,000,000","-2,916,000,000",NaN,NaN,"5,964,000,000","-19,594,000,000","25,644,000,000",2023-03-31
12/31/2022,"34,005,000,000","34,005,000,000","29,998,000,000",NaN,NaN,NaN,NaN,"2,916,000,000","2,916,000,000",NaN,NaN,NaN,"2,905,000,000",NaN,"-317,000,000","-1,497,000,000","6,595,000,000","4,275,000,000","-1,807,000,000",NaN,"-6,075,000,000","-6,075,000,000",NaN,NaN,"-6,075,000,000",NaN,"-4,099,000,000","3,758,000,000","131,000,000","-1,445,000,000","-1,445,000,000","-3,787,000,000","-3,787,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"2,483,000,000","-5,153,000,000","7,636,000,000","-141,000,000","-35,563,000,000","-35,563,000,000","-9,615,000,000","-1,401,000,000",NaN,"-1,401,000,000","-8,214,000,000",NaN,"-8,214,000,000","-19,475,000,000",NaN,"-19,475,000,000","-3,768,000,000","-3,768,000,000",NaN,"-2,705,000,000","21,974,000,000","-3,003,000,000","24,977,000,000",NaN,"828,000,000","703,000,000","-3,787,000,000",NaN,NaN,"-9,615,000,000","-19,475,000,000","30,218,000,000",2022-12-31
09/30/2022,"24,127,000,000","24,127,000,000","20,721,000,000",NaN,NaN,NaN,NaN,"2,865,000,000","2,865,000,000","-1,861,000,000","-1,861,000,000",NaN,"2,278,000,000",NaN,"172,000,000","-48,000,000","-18,693,000,000","-6,384,

In [31]:
# Set aapl_cashflow_df with 'Date' as index
aapl_cashflow_df = aapl_cashflow_df.set_index('Date')

# View aapl_cashflow_df to make sure 'Date' is now the index
aapl_cashflow_df.head()

name,OperatingCashFlow,\tCashFlowFromContinuingOperatingActivities,\t\tNetIncomeFromContinuingOperations,\t\tOperatingGainsLosses,\t\t\tGainLossOnSaleOfBusiness,\t\t\tGainLossOnSaleOfPPE,\t\t\tGainLossOnInvestmentSecurities,\t\tDepreciationAmortizationDepletion,\t\t\tDepreciationAndAmortization,\t\tDeferredTax,\t\t\tDeferredIncomeTax,\t\tUnrealizedGainLossOnInvestmentSecurities,\t\tStockBasedCompensation,\t\tExcessTaxBenefitFromStockBasedCompensation,\t\tOtherNonCashItems,\t\tChangeInWorkingCapital,\t\t\tChangeInReceivables,\t\t\t\tChangesInAccountReceivables,\t\t\tChangeInInventory,\t\t\tChangeInPrepaidAssets,\t\t\tChangeInPayablesAndAccruedExpense,\t\t\t\tChangeInPayable,\t\t\t\t\tChangeInTaxPayable,\t\t\t\t\t\tChangeInIncomeTaxPayable,\t\t\t\t\tChangeInAccountPayable,\t\t\t\tChangeInAccruedExpense,\t\t\tChangeInOtherCurrentAssets,\t\t\tChangeInOtherCurrentLiabilities,\t\t\tChangeInOtherWorkingCapital,InvestingCashFlow,\tCashFlowFromContinuingInvestingActivities,\t\tNetPPEPurchaseAndSale,\t\t\tPurchaseOfPPE,\t\t\tSaleOfPPE,\t\tNetIntangiblesPurchaseAndSale,\t\t\tPurchaseOfIntangibles,\t\tNetBusinessPurchaseAndSale,\t\t\tPurchaseOfBusiness,\t\t\tSaleOfBusiness,\t\tNetInvestmentPurchaseAndSale,\t\t\tPurchaseOfInvestment,\t\t\tSaleOfInvestment,\t\tNetOtherInvestingChanges,FinancingCashFlow,\tCashFlowFromContinuingFinancingActivities,\t\tNetIssuancePaymentsOfDebt,\t\t\tNetLongTermDebtIssuance,\t\t\t\tLongTermDebtIssuance,\t\t\t\tLongTermDebtPayments,\t\t\tNetShortTermDebtIssuance,\t\t\t\tShortTermDebtIssuance,\t\t\t\tShortTermDebtPayments,\t\tNetCommonStockIssuance,\t\t\tCommonStockIssuance,\t\t\tCommonStockPayments,\t\tCashDividendsPaid,\t\t\tCommonStockDividendPaid,\t\tProceedsFromStockOptionExercised,\t\tNetOtherFinancingCharges,EndCashPosition,\tChangesInCash,\tBeginningCashPosition,\tOtherCashAdjustmentOutsideChangeinCash,IncomeTaxPaidSupplementalData,InterestPaidSupplementalData,CapitalExpenditure,IssuanceOfCapitalStock,IssuanceOfDebt,RepaymentOfDebt,RepurchaseOfCapitalStock,FreeCashFlow
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-31,"28,560,000,000","28,560,000,000","24,160,000,000",NaN,NaN,NaN,NaN,"2,898,000,000","2,898,000,000",NaN,NaN,NaN,"2,686,000,000",NaN,"-1,415,000,000","231,000,000","17,786,000,000","5,321,000,000","-741,000,000",NaN,"-14,689,000,000","-14,689,000,000",NaN,NaN,"-14,689,000,000",NaN,"7,000,000","-2,001,000,000",NaN,"2,319,000,000","2,319,000,000","-2,916,000,000","-2,916,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"5,341,000,000","-6,044,000,000","11,385,000,000","-106,000,000","-25,724,000,000","-25,724,000,000","-1,996,000,000","-2,250,000,000",NaN,"-2,250,000,000","254,000,000",NaN,NaN,"-19,594,000,000",NaN,"-19,594,000,000","-3,650,000,000","-3,650,000,000",NaN,"-484,000,000","27,129,000,000","5,155,000,000","21,974,000,000",NaN,"4,066,000,000","1,170,000,000","-2,916,000,000",NaN,NaN,"5,964,000,000","-19,594,000,000","25,644,000,000"
2022-12-31,"34,005,000,000","34,005,000,000","29,998,000,000",NaN,NaN,NaN,NaN,"2,916,000,000","2,916,000,000",NaN,NaN,NaN,"2,905,000,000",NaN,"-317,000,000","-1,497,000,000","6,595,000,000","4,275,000,000","-1,807,000,000",NaN,"-6,075,000,000","-6,075,000,000",NaN,NaN,"-6,075,000,000",NaN,"-4,099,000,000","3,758,000,000","131,000,000","-1,445,000,000","-1,445,000,000","-3,787,000,000","-3,787,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"2,483,000,000","-5,153,000,000","7,636,000,000","-141,000,000","-35,563,000,000","-35,563,000,000","-9,615,000,000","-1,401,000,000",NaN,"-1,401,000,000","-8,214,000,000",NaN,"-8,214,000,000","-19,475,000,000",NaN,"-19,475,000,000","-3,768,000,000","-3,768,000,000",NaN,"-2,705,000,000","21,974,000,000","-3,003,000,000","24,977,000,000",NaN,"828,000,000","703,000,000","-3,787,000,000",NaN,NaN,"-9,615,000,000","-19,475,000,000","30,218,000,000"
2022-09-30,"24,127,000,000","24,127,000,000","20,721,000,000",NaN,NaN,NaN,NaN,"2,865,000,000","2,865,000,000","-1,861,000,000","-1,861,000,000",NaN,"2,278,000,000",NaN,"172,

In [32]:
# Balance Sheet for AAPL
aapl_balance_df = pd.read_csv(
    Path("AAPL_quarterly_balance-sheet.csv"), 
     index_col = 'name',
    parse_dates=True,
    infer_datetime_format=True).T

# View aapl_balance_df
aapl_balance_df.head()

name,TotalAssets,\tCurrentAssets,\t\tCashCashEquivalentsAndShortTermInvestments,\t\t\tCashAndCashEquivalents,\t\t\t\tCashFinancial,\t\t\t\tCashEquivalents,\t\t\tOtherShortTermInvestments,\t\tReceivables,\t\t\tAccountsReceivable,\t\t\t\tGrossAccountsReceivable,\t\t\t\tAllowanceForDoubtfulAccountsReceivable,\t\t\tOtherReceivables,\t\t\tReceivablesAdjustmentsAllowances,\t\tInventory,\t\t\tRawMaterials,\t\t\tWorkInProcess,\t\t\tFinishedGoods,\t\t\tOtherInventories,\t\tPrepaidAssets,\t\tCurrentDeferredAssets,\t\t\tCurrentDeferredTaxesAssets,\t\tOtherCurrentAssets,\tTotalNonCurrentAssets,\t\tNetPPE,\t\t\tGrossPPE,\t\t\t\tProperties,\t\t\t\tLandAndImprovements,\t\t\t\tBuildingsAndImprovements,\t\t\t\tMachineryFurnitureEquipment,\t\t\t\tOtherProperties,\t\t\t\tLeases,\t\t\tAccumulatedDepreciation,\t\tGoodwillAndOtherIntangibleAssets,\t\t\tGoodwill,\t\t\tOtherIntangibleAssets,\t\tInvestmentsAndAdvances,\t\t\tInvestmentinFinancialAssets,\t\t\t\tAvailableForSaleSecurities,\t\t\tOtherInvestments,\t\tNonCurrentDeferredAssets,\t\t\tNonCurrentDeferredTaxesAssets,\t\tNonCurrentPrepaidAssets,\t\tOtherNonCurrentAssets,TotalLiabilitiesNetMinorityInterest,\tCurrentLiabilities,\t\tPayablesAndAccruedExpenses,\t\t\tPayables,\t\t\t\tAccountsPayable,\t\t\t\tTotalTaxPayable,\t\t\t\t\tIncomeTaxPayable,\t\t\t\tOtherPayable,\t\t\tCurrentAccruedExpenses,\t\tPensionandOtherPostRetirementBenefitPlansCurrent,\t\tCurrentDebtAndCapitalLeaseObligation,\t\t\tCurrentDebt,\t\t\t\tCurrentNotesPayable,\t\t\t\tCommercialPaper,\t\t\t\tOtherCurrentBorrowings,\t\tCurrentDeferredLiabilities,\t\t\tCurrentDeferredTaxesLiabilities,\t\t\tCurrentDeferredRevenue,\t\tOtherCurrentLiabilities,\tTotalNonCurrentLiabilitiesNetMinorityInterest,\t\tLongTermDebtAndCapitalLeaseObligation,\t\t\tLongTermDebt,\t\tNonCurrentDeferredLiabilities,\t\t\tNonCurrentDeferredTaxesLiabilities,\t\t\tNonCurrentDeferredRevenue,\t\tTradeandOtherPayablesNonCurrent,\t\tOtherNonCurrentLiabilities,TotalEquityGrossMinorityInterest,\tStockholdersEquity,\t\tCapitalStock,\t\t\tPreferredStock,\t\t\tCommonStock,\t\tAdditionalPaidInCapital,\t\tRetainedEarnings,\t\tGainsLossesNotAffectingRetainedEarnings,\t\t\tForeignCurrencyTranslationAdjustments,\t\t\tOtherEquityAdjustments,\t\tOtherEquityInterest,TotalCapitalization,PreferredStockEquity,CommonStockEquity,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,NetDebt,ShareIssued,OrdinarySharesNumber
03/31/2023,"332,160,000,000","112,913,000,000","55,872,000,000","24,687,000,000","20,050,000,000","4,637,000,000","31,185,000,000","35,899,000,000","17,936,000,000",NaN,NaN,"17,963,000,000",NaN,"7,482,000,000","3,379,000,000",NaN,"4,103,000,000",NaN,NaN,NaN,NaN,"13,660,000,000","219,247,000,000","43,398,000,000","113,066,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-69,668,000,000",NaN,NaN,NaN,"110,461,000,000","110,461,000,000","110,461,000,000",NaN,NaN,NaN,NaN,"65,388,000,000","270,002,000,000","120,075,000,000","42,945,000,000","42,945,000,000","42,945,000,000",NaN,NaN,NaN,NaN,NaN,"12,574,000,000","12,574,000,000",NaN,"1,996,000,000","10,578,000,000","8,131,000,000",NaN,"8,131,000,000","56,425,000,000","149,927,000,000","97,041,000,000","97,041,000,000",NaN,NaN,NaN,NaN,"52,886,000,000","62,158,000,000","62,158,000,000","69,568,000,000",NaN,"69,568,000,000",NaN,"4,336,000,000","-11,746,000,000",NaN,"-11,746,000,000",NaN,"159,199,000,000",NaN,"62,158,000,000","62,158,000,000","-7,162,000,000","171,773,000,000","62,158,000,000","109,615,000,000","84,928,000,000","15,723,406,000","15,723,406,000"
12/31/2022,"346,747,000,000","128,777,000,000","51,355,000,000","20,535,000,000","17,908,000,000","2,627,000,000","30,820,000,000","54,180,000,000","23,752,000,000",NaN,NaN,"30,428,000,000",NaN,"6,820,000,000","2,513,000,000",NaN,"4,307,000,000",NaN,NaN,NaN,NaN,"16,422,000,000","217,970,000,000","42,951,000,000","110,995,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-68,044,000,000",NaN,NaN,NaN,"114,095,000,000",NaN,NaN,"114,095,000,000",NaN,NaN,NaN,"60,924,000,000","290,020,000

In [33]:
# Create a new column 'Date' that takes in the datetime format 
aapl_balance_df['Date'] = pd.to_datetime(aapl_balance_df.index).strftime('%Y-%m-%d')

# View aapl_balance_df to make sure that 'Date' has been created as a new column
aapl_balance_df.head()

name,TotalAssets,\tCurrentAssets,\t\tCashCashEquivalentsAndShortTermInvestments,\t\t\tCashAndCashEquivalents,\t\t\t\tCashFinancial,\t\t\t\tCashEquivalents,\t\t\tOtherShortTermInvestments,\t\tReceivables,\t\t\tAccountsReceivable,\t\t\t\tGrossAccountsReceivable,\t\t\t\tAllowanceForDoubtfulAccountsReceivable,\t\t\tOtherReceivables,\t\t\tReceivablesAdjustmentsAllowances,\t\tInventory,\t\t\tRawMaterials,\t\t\tWorkInProcess,\t\t\tFinishedGoods,\t\t\tOtherInventories,\t\tPrepaidAssets,\t\tCurrentDeferredAssets,\t\t\tCurrentDeferredTaxesAssets,\t\tOtherCurrentAssets,\tTotalNonCurrentAssets,\t\tNetPPE,\t\t\tGrossPPE,\t\t\t\tProperties,\t\t\t\tLandAndImprovements,\t\t\t\tBuildingsAndImprovements,\t\t\t\tMachineryFurnitureEquipment,\t\t\t\tOtherProperties,\t\t\t\tLeases,\t\t\tAccumulatedDepreciation,\t\tGoodwillAndOtherIntangibleAssets,\t\t\tGoodwill,\t\t\tOtherIntangibleAssets,\t\tInvestmentsAndAdvances,\t\t\tInvestmentinFinancialAssets,\t\t\t\tAvailableForSaleSecurities,\t\t\tOtherInvestments,\t\tNonCurrentDeferredAssets,\t\t\tNonCurrentDeferredTaxesAssets,\t\tNonCurrentPrepaidAssets,\t\tOtherNonCurrentAssets,TotalLiabilitiesNetMinorityInterest,\tCurrentLiabilities,\t\tPayablesAndAccruedExpenses,\t\t\tPayables,\t\t\t\tAccountsPayable,\t\t\t\tTotalTaxPayable,\t\t\t\t\tIncomeTaxPayable,\t\t\t\tOtherPayable,\t\t\tCurrentAccruedExpenses,\t\tPensionandOtherPostRetirementBenefitPlansCurrent,\t\tCurrentDebtAndCapitalLeaseObligation,\t\t\tCurrentDebt,\t\t\t\tCurrentNotesPayable,\t\t\t\tCommercialPaper,\t\t\t\tOtherCurrentBorrowings,\t\tCurrentDeferredLiabilities,\t\t\tCurrentDeferredTaxesLiabilities,\t\t\tCurrentDeferredRevenue,\t\tOtherCurrentLiabilities,\tTotalNonCurrentLiabilitiesNetMinorityInterest,\t\tLongTermDebtAndCapitalLeaseObligation,\t\t\tLongTermDebt,\t\tNonCurrentDeferredLiabilities,\t\t\tNonCurrentDeferredTaxesLiabilities,\t\t\tNonCurrentDeferredRevenue,\t\tTradeandOtherPayablesNonCurrent,\t\tOtherNonCurrentLiabilities,TotalEquityGrossMinorityInterest,\tStockholdersEquity,\t\tCapitalStock,\t\t\tPreferredStock,\t\t\tCommonStock,\t\tAdditionalPaidInCapital,\t\tRetainedEarnings,\t\tGainsLossesNotAffectingRetainedEarnings,\t\t\tForeignCurrencyTranslationAdjustments,\t\t\tOtherEquityAdjustments,\t\tOtherEquityInterest,TotalCapitalization,PreferredStockEquity,CommonStockEquity,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,NetDebt,ShareIssued,OrdinarySharesNumber,Date
03/31/2023,"332,160,000,000","112,913,000,000","55,872,000,000","24,687,000,000","20,050,000,000","4,637,000,000","31,185,000,000","35,899,000,000","17,936,000,000",NaN,NaN,"17,963,000,000",NaN,"7,482,000,000","3,379,000,000",NaN,"4,103,000,000",NaN,NaN,NaN,NaN,"13,660,000,000","219,247,000,000","43,398,000,000","113,066,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-69,668,000,000",NaN,NaN,NaN,"110,461,000,000","110,461,000,000","110,461,000,000",NaN,NaN,NaN,NaN,"65,388,000,000","270,002,000,000","120,075,000,000","42,945,000,000","42,945,000,000","42,945,000,000",NaN,NaN,NaN,NaN,NaN,"12,574,000,000","12,574,000,000",NaN,"1,996,000,000","10,578,000,000","8,131,000,000",NaN,"8,131,000,000","56,425,000,000","149,927,000,000","97,041,000,000","97,041,000,000",NaN,NaN,NaN,NaN,"52,886,000,000","62,158,000,000","62,158,000,000","69,568,000,000",NaN,"69,568,000,000",NaN,"4,336,000,000","-11,746,000,000",NaN,"-11,746,000,000",NaN,"159,199,000,000",NaN,"62,158,000,000","62,158,000,000","-7,162,000,000","171,773,000,000","62,158,000,000","109,615,000,000","84,928,000,000","15,723,406,000","15,723,406,000",2023-03-31
12/31/2022,"346,747,000,000","128,777,000,000","51,355,000,000","20,535,000,000","17,908,000,000","2,627,000,000","30,820,000,000","54,180,000,000","23,752,000,000",NaN,NaN,"30,428,000,000",NaN,"6,820,000,000","2,513,000,000",NaN,"4,307,000,000",NaN,NaN,NaN,NaN,"16,422,000,000","217,970,000,000","42,951,000,000","110,995,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-68,044,000,000",NaN,NaN,NaN,"114,095,000,000",NaN,NaN,"114,095,000,000",NaN,NaN,NaN,"60,924,000,0

In [34]:
# Set aapl_balance_df with 'Date' as index
aapl_balance_df = aapl_balance_df.set_index('Date')

# View aapl_balance_df to make sure 'Date' is now the index
aapl_balance_df.head()

name,TotalAssets,\tCurrentAssets,\t\tCashCashEquivalentsAndShortTermInvestments,\t\t\tCashAndCashEquivalents,\t\t\t\tCashFinancial,\t\t\t\tCashEquivalents,\t\t\tOtherShortTermInvestments,\t\tReceivables,\t\t\tAccountsReceivable,\t\t\t\tGrossAccountsReceivable,\t\t\t\tAllowanceForDoubtfulAccountsReceivable,\t\t\tOtherReceivables,\t\t\tReceivablesAdjustmentsAllowances,\t\tInventory,\t\t\tRawMaterials,\t\t\tWorkInProcess,\t\t\tFinishedGoods,\t\t\tOtherInventories,\t\tPrepaidAssets,\t\tCurrentDeferredAssets,\t\t\tCurrentDeferredTaxesAssets,\t\tOtherCurrentAssets,\tTotalNonCurrentAssets,\t\tNetPPE,\t\t\tGrossPPE,\t\t\t\tProperties,\t\t\t\tLandAndImprovements,\t\t\t\tBuildingsAndImprovements,\t\t\t\tMachineryFurnitureEquipment,\t\t\t\tOtherProperties,\t\t\t\tLeases,\t\t\tAccumulatedDepreciation,\t\tGoodwillAndOtherIntangibleAssets,\t\t\tGoodwill,\t\t\tOtherIntangibleAssets,\t\tInvestmentsAndAdvances,\t\t\tInvestmentinFinancialAssets,\t\t\t\tAvailableForSaleSecurities,\t\t\tOtherInvestments,\t\tNonCurrentDeferredAssets,\t\t\tNonCurrentDeferredTaxesAssets,\t\tNonCurrentPrepaidAssets,\t\tOtherNonCurrentAssets,TotalLiabilitiesNetMinorityInterest,\tCurrentLiabilities,\t\tPayablesAndAccruedExpenses,\t\t\tPayables,\t\t\t\tAccountsPayable,\t\t\t\tTotalTaxPayable,\t\t\t\t\tIncomeTaxPayable,\t\t\t\tOtherPayable,\t\t\tCurrentAccruedExpenses,\t\tPensionandOtherPostRetirementBenefitPlansCurrent,\t\tCurrentDebtAndCapitalLeaseObligation,\t\t\tCurrentDebt,\t\t\t\tCurrentNotesPayable,\t\t\t\tCommercialPaper,\t\t\t\tOtherCurrentBorrowings,\t\tCurrentDeferredLiabilities,\t\t\tCurrentDeferredTaxesLiabilities,\t\t\tCurrentDeferredRevenue,\t\tOtherCurrentLiabilities,\tTotalNonCurrentLiabilitiesNetMinorityInterest,\t\tLongTermDebtAndCapitalLeaseObligation,\t\t\tLongTermDebt,\t\tNonCurrentDeferredLiabilities,\t\t\tNonCurrentDeferredTaxesLiabilities,\t\t\tNonCurrentDeferredRevenue,\t\tTradeandOtherPayablesNonCurrent,\t\tOtherNonCurrentLiabilities,TotalEquityGrossMinorityInterest,\tStockholdersEquity,\t\tCapitalStock,\t\t\tPreferredStock,\t\t\tCommonStock,\t\tAdditionalPaidInCapital,\t\tRetainedEarnings,\t\tGainsLossesNotAffectingRetainedEarnings,\t\t\tForeignCurrencyTranslationAdjustments,\t\t\tOtherEquityAdjustments,\t\tOtherEquityInterest,TotalCapitalization,PreferredStockEquity,CommonStockEquity,NetTangibleAssets,WorkingCapital,InvestedCapital,TangibleBookValue,TotalDebt,NetDebt,ShareIssued,OrdinarySharesNumber
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-31,"332,160,000,000","112,913,000,000","55,872,000,000","24,687,000,000","20,050,000,000","4,637,000,000","31,185,000,000","35,899,000,000","17,936,000,000",NaN,NaN,"17,963,000,000",NaN,"7,482,000,000","3,379,000,000",NaN,"4,103,000,000",NaN,NaN,NaN,NaN,"13,660,000,000","219,247,000,000","43,398,000,000","113,066,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-69,668,000,000",NaN,NaN,NaN,"110,461,000,000","110,461,000,000","110,461,000,000",NaN,NaN,NaN,NaN,"65,388,000,000","270,002,000,000","120,075,000,000","42,945,000,000","42,945,000,000","42,945,000,000",NaN,NaN,NaN,NaN,NaN,"12,574,000,000","12,574,000,000",NaN,"1,996,000,000","10,578,000,000","8,131,000,000",NaN,"8,131,000,000","56,425,000,000","149,927,000,000","97,041,000,000","97,041,000,000",NaN,NaN,NaN,NaN,"52,886,000,000","62,158,000,000","62,158,000,000","69,568,000,000",NaN,"69,568,000,000",NaN,"4,336,000,000","-11,746,000,000",NaN,"-11,746,000,000",NaN,"159,199,000,000",NaN,"62,158,000,000","62,158,000,000","-7,162,000,000","171,773,000,000","62,158,000,000","109,615,000,000","84,928,000,000","15,723,406,000","15,723,406,000"
2022-12-31,"346,747,000,000","128,777,000,000","51,355,000,000","20,535,000,000","17,908,000,000","2,627,000,000","30,820,000,000","54,180,000,000","23,752,000,000",NaN,NaN,"30,428,000,000",NaN,"6,820,000,000","2,513,000,000",NaN,"4,307,000,000",NaN,NaN,NaN,NaN,"16,422,000,000","217,970,000,000","42,951,000,000","110,995,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"-68,044,000,000",

In [35]:
# Find CashFlow, Outstanding Shares, Total Debt and assign them a respective variable name
aapl_cash = aapl_cashflow_df['FreeCashFlow'] # Cash
aapl_debt = aapl_balance_df['TotalDebt'] # Debt
aapl_shares = aapl_balance_df['ShareIssued'] # Shares

# Make sure that variables are given correct data
display(aapl_cash.head())
display(aapl_debt.head())
display(aapl_shares.head())

Date
2023-03-31    25,644,000,000
2022-12-31    30,218,000,000
2022-09-30    20,838,000,000
2022-06-30    20,790,000,000
2022-03-31    25,652,000,000
Name: FreeCashFlow, dtype: object

Date
2023-03-31    109,615,000,000
2022-12-31    111,110,000,000
2022-09-30    120,069,000,000
2022-06-30    119,691,000,000
2022-03-31    119,981,000,000
Name: TotalDebt, dtype: object

Date
2023-03-31    15,723,406,000
2022-12-31    15,842,407,000
2022-09-30    15,943,425,000
2022-06-30    16,095,378,000
2022-03-31    16,207,568,000
Name: ShareIssued, dtype: object

## Part 2: Grab ticker and Quarterly Date Frame from yfinance

### AAPL_1: "03/32/2023" to "07/03/2023"

In [36]:
# Pull aapl ticker with date range "03/32/2023" to "07/03/2023"
aapl_1 = yf.download("AAPL", start='2023-03-31', end='2023-07-03', interval="1d")

# View aapl_1
aapl_1.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-03-31,162.440002,165.000000,161.910004,164.899994,164.672226,68749800
2023-04-03,164.270004,166.289993,164.220001,166.169998,165.940475,56976200
2023-04-04,166.600006,166.839996,165.110001,165.630005,165.401230,46278300
2023-04-05,164.740005,165.050003,161.800003,163.759995,163.533798,51511700
2023-04-06,162.429993,164.960007,162.000000,164.660004,164.432556,45390100


In [37]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_1['Total Debt'] = aapl_debt.loc['2023-03-31']
aapl_1['Shares'] = aapl_shares.loc['2023-03-31']
aapl_1['Cash'] = aapl_cash.loc['2023-03-31']
aapl_1['EPS'] = 1.53
aapl_1['EBITDA'] = 31260000000

# Check aapl_1 to make sure data is correct        
aapl_1.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2023-03-31,162.440002,165.000000,161.910004,164.899994,164.672226,68749800,"109,615,000,000","15,723,406,000","25,644,000,000",1.53,31260000000
2023-04-03,164.270004,166.289993,164.220001,166.169998,165.940475,56976200,"109,615,000,000","15,723,406,000","25,644,000,000",1.53,31260000000
2023-04-04,166.600006,166.839996,165.110001,165.630005,165.401230,46278300,"109,615,000,000","15,723,406,000","25,644,000,000",1.53,31260000000
2023-04-05,164.740005,165.050003,161.800003,163.759995,163.533798,51511700,"109,615,000,000","15,723,406,000","25,644,000,000",1.53,31260000000
2023-04-06,162.429993,164.960007,162.000000,164.660004,164.432556,45390100,"109,615,000,000","15,723,406,000","25,644,000,000",1.53,31260000000


### AAPL_2: "12-31-2022" to "03/30/2023"

In [38]:
# Pull aapl ticker with date range "12/31/2022" to "03/30/2023"
aapl_2 = yf.download("AAPL", start='2022-12-31', end='2023-03-30', interval="1d")

# View aapl_2
aapl_2.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,124.706833,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.993095,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,124.656975,80962700
2023-01-06,126.010002,130.289993,124.889999,129.619995,129.243622,87754700
2023-01-09,130.470001,133.410004,129.889999,130.149994,129.772079,70790800


In [39]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_2['Total Debt'] = aapl_debt.loc['2022-12-31']
aapl_2['Shares'] = aapl_shares.loc['2022-12-31']
aapl_2['Cash'] = aapl_cash.loc['2022-12-31']
aapl_2['EPS'] = 1.89
aapl_2['EBITDA'] = 38932000000

# Check aapl_2 to make sure data is correct        
aapl_2.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,124.706833,112117500,"111,110,000,000","15,842,407,000","30,218,000,000",1.89,38932000000
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.993095,89113600,"111,110,000,000","15,842,407,000","30,218,000,000",1.89,38932000000
2023-01-05,127.129997,127.769997,124.760002,125.019997,124.656975,80962700,"111,110,000,000","15,842,407,000","30,218,000,000",1.89,38932000000
2023-01-06,126.010002,130.289993,124.889999,129.619995,129.243622,87754700,"111,110,000,000","15,842,407,000","30,218,000,000",1.89,38932000000
2023-01-09,130.470001,133.410004,129.889999,130.149994,129.772079,70790800,"111,110,000,000","15,842,407,000","30,218,000,000",1.89,38932000000


#### AAPL_3: "09/30/2022" to "12/30/2022"

In [40]:
# Pull aapl ticker with date range "09/30/2022" to "12/30/2022"
aapl_3 = yf.download("AAPL", start='2022-09-30', end = '2022-12-30', interval="1d")

# Check aapl_3 to make sure data is correct
aapl_3.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.570480,124925300
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.801132,114311700
2022-10-04,145.029999,146.220001,144.259995,146.100006,145.434525,87830100
2022-10-05,144.070007,147.380005,143.009995,146.399994,145.733124,79471000
2022-10-06,145.809998,147.539993,145.220001,145.429993,144.767563,68402200


In [41]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_3['Total Debt'] = aapl_debt.loc['2022-09-30']
aapl_3['Shares'] = aapl_shares.loc['2022-09-30']
aapl_3['Cash'] = aapl_cash.loc['2022-09-30']
aapl_3['EPS'] = 1.29
aapl_3['EBITDA'] = 27759000000

# Check to make sure aapl_3 is correct
aapl_3.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.570480,124925300,"120,069,000,000","15,943,425,000","20,838,000,000",1.29,27759000000
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.801132,114311700,"120,069,000,000","15,943,425,000","20,838,000,000",1.29,27759000000
2022-10-04,145.029999,146.220001,144.259995,146.100006,145.434525,87830100,"120,069,000,000","15,943,425,000","20,838,000,000",1.29,27759000000
2022-10-05,144.070007,147.380005,143.009995,146.399994,145.733124,79471000,"120,069,000,000","15,943,425,000","20,838,000,000",1.29,27759000000
2022-10-06,145.809998,147.539993,145.220001,145.429993,144.767563,68402200,"120,069,000,000","15,943,425,000","20,838,000,000",1.29,27759000000


### AAPL_4: "06/30/2022" to "09/29/2022"

In [42]:
# Pull aapl ticker with date range "06/30/2022" to "09/29/2022"
aapl_4 = yf.download("AAPL", start='2022-06-30', end = '2022-09-29', interval="1d")

# View aapl_4
aapl_4.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-06-30,137.250000,138.369995,133.770004,136.720001,135.908463,98964500
2022-07-01,136.039993,139.039993,135.660004,138.929993,138.105331,71051600
2022-07-05,137.770004,141.610001,136.929993,141.559998,140.719727,73353800
2022-07-06,141.350006,144.119995,141.080002,142.919998,142.071655,74064300
2022-07-07,143.289993,146.550003,143.279999,146.350006,145.481293,66253700


In [43]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_4['Total Debt'] = aapl_debt.loc['2022-06-30']
aapl_4['Shares'] = aapl_shares.loc['2022-06-30']
aapl_4['Cash'] = aapl_cash.loc['2022-06-30']
aapl_4['EPS'] = 1.20
aapl_4['EBITDA'] = 25881000000

# Check to make sure aapl_3 is correct
aapl_4.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2022-06-30,137.250000,138.369995,133.770004,136.720001,135.908463,98964500,"119,691,000,000","16,095,378,000","20,790,000,000",1.2,25881000000
2022-07-01,136.039993,139.039993,135.660004,138.929993,138.105331,71051600,"119,691,000,000","16,095,378,000","20,790,000,000",1.2,25881000000
2022-07-05,137.770004,141.610001,136.929993,141.559998,140.719727,73353800,"119,691,000,000","16,095,378,000","20,790,000,000",1.2,25881000000
2022-07-06,141.350006,144.119995,141.080002,142.919998,142.071655,74064300,"119,691,000,000","16,095,378,000","20,790,000,000",1.2,25881000000
2022-07-07,143.289993,146.550003,143.279999,146.350006,145.481293,66253700,"119,691,000,000","16,095,378,000","20,790,000,000",1.2,25881000000


### AAPL_5: "03/31/2022" to "06/29/2022"

In [46]:
# Pull aapl ticker with date range "03/31/2022" to "06/29/2022"
aapl_5 = yf.download("AAPL", start='2022-03-31', end = '2022-06-29', interval="1d")

# Check first five rows 
aapl_5.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-31,177.839996,178.029999,174.399994,174.610001,173.318909,103049300
2022-04-01,174.029999,174.880005,171.940002,174.309998,173.021118,78751300
2022-04-04,174.570007,178.490005,174.440002,178.440002,177.120590,76468400
2022-04-05,177.500000,178.300003,174.419998,175.059998,173.765594,73401800
2022-04-06,172.360001,173.630005,170.130005,171.830002,170.559464,89058800


In [47]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_5['Total Debt'] = aapl_debt.loc['2022-03-31']
aapl_5['Shares'] = aapl_shares.loc['2022-03-31']
aapl_5['Cash'] = aapl_cash.loc['2022-03-31']
aapl_5['EPS'] = 1.54
aapl_5['EBITDA'] = 32716000000

# View first 5 rows in data
aapl_5.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2022-03-31,177.839996,178.029999,174.399994,174.610001,173.318909,103049300,"119,981,000,000","16,207,568,000","25,652,000,000",1.54,32716000000
2022-04-01,174.029999,174.880005,171.940002,174.309998,173.021118,78751300,"119,981,000,000","16,207,568,000","25,652,000,000",1.54,32716000000
2022-04-04,174.570007,178.490005,174.440002,178.440002,177.120590,76468400,"119,981,000,000","16,207,568,000","25,652,000,000",1.54,32716000000
2022-04-05,177.500000,178.300003,174.419998,175.059998,173.765594,73401800,"119,981,000,000","16,207,568,000","25,652,000,000",1.54,32716000000
2022-04-06,172.360001,173.630005,170.130005,171.830002,170.559464,89058800,"119,981,000,000","16,207,568,000","25,652,000,000",1.54,32716000000


### AAPL_6: "12/31/2021" to "03/30/2022"

In [48]:
# Pull aapl ticker with date range "12/31/2021" to "03/30/2022"
aapl_6 = yf.download("AAPL", start='2021-12-31', end = '2022-03-30', interval="1d")

# View first five rows in data
aapl_6.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,178.089996,179.229996,177.259995,177.570007,176.032761,64062300
2022-01-03,177.830002,182.880005,177.710007,182.009995,180.434311,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,178.144272,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,173.405685,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,170.510941,96904000


In [50]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_6['Total Debt'] = aapl_debt.loc['2021-12-31']
aapl_6['Shares'] = aapl_shares.loc['2021-12-31']
aapl_6['Cash'] = aapl_cash.loc['2021-12-31']
aapl_6['EPS'] = 2.11
aapl_6['EBITDA'] = 44185000000

# View first five rows in data
aapl_6.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2021-12-31,178.089996,179.229996,177.259995,177.570007,176.032761,64062300,"122,798,000,000","16,334,371,000","44,163,000,000",2.11,44185000000
2022-01-03,177.830002,182.880005,177.710007,182.009995,180.434311,104487900,"122,798,000,000","16,334,371,000","44,163,000,000",2.11,44185000000
2022-01-04,182.630005,182.940002,179.119995,179.699997,178.144272,99310400,"122,798,000,000","16,334,371,000","44,163,000,000",2.11,44185000000
2022-01-05,179.610001,180.169998,174.639999,174.919998,173.405685,94537600,"122,798,000,000","16,334,371,000","44,163,000,000",2.11,44185000000
2022-01-06,172.699997,175.300003,171.639999,172.000000,170.510941,96904000,"122,798,000,000","16,334,371,000","44,163,000,000",2.11,44185000000


### AAPL_7: "09/30/2021" to "12/30/2021"

In [53]:
# Pull aapl ticker with date range "09/30/2021" to "12/30/2021"
aapl_7 = yf.download("AAPL", start='2021-09-30', end = '2021-12-30', interval="1d")

# View first 5 rows of the data
aapl_7.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-30,143.660004,144.380005,141.279999,141.500000,140.070587,89056700
2021-10-01,141.899994,142.919998,139.110001,142.649994,141.208954,94639600
2021-10-04,141.759995,142.210007,138.270004,139.139999,137.734421,98322000
2021-10-05,139.490005,142.240005,139.360001,141.110001,139.684525,80861100
2021-10-06,139.470001,142.149994,138.369995,142.000000,140.565521,83221100


In [54]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_7['Total Debt'] = aapl_debt.loc['2021-09-30']
aapl_7['Shares'] = aapl_shares.loc['2021-09-30']
aapl_7['Cash'] = aapl_cash.loc['2021-09-30']
aapl_7['EPS'] = 1.25
aapl_7['EBITDA'] = 26775000000

# View first five rows of data
aapl_7.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2021-09-30,143.660004,144.380005,141.279999,141.500000,140.070587,89056700,"124,719,000,000","16,426,786,000","16,977,000,000",1.25,26775000000
2021-10-01,141.899994,142.919998,139.110001,142.649994,141.208954,94639600,"124,719,000,000","16,426,786,000","16,977,000,000",1.25,26775000000
2021-10-04,141.759995,142.210007,138.270004,139.139999,137.734421,98322000,"124,719,000,000","16,426,786,000","16,977,000,000",1.25,26775000000
2021-10-05,139.490005,142.240005,139.360001,141.110001,139.684525,80861100,"124,719,000,000","16,426,786,000","16,977,000,000",1.25,26775000000
2021-10-06,139.470001,142.149994,138.369995,142.000000,140.565521,83221100,"124,719,000,000","16,426,786,000","16,977,000,000",1.25,26775000000


### AAPL_8: "07/05/2021" to "09/29/2021"

In [57]:
# Pull aapl ticker with date range "07/05/2021" to "09/29/2021"
aapl_8 = yf.download("AAPL", start='2021-07-05', end = '2021-09-29', interval="1d")

# View first 5 rows of data
aapl_8.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-06,140.070007,143.149994,140.070007,142.020004,140.375000,108181800
2021-07-07,143.539993,144.889999,142.660004,144.570007,142.895493,104911600
2021-07-08,141.580002,144.059998,140.669998,143.240005,141.580872,105575500
2021-07-09,142.750000,145.649994,142.649994,145.110001,143.429230,99890800
2021-07-12,146.210007,146.320007,144.000000,144.500000,142.826279,76299700


In [58]:
# Create 'Total Debt', 'Shares', 'Cash', 'EPS', 'EBITDA' and concat this data to yfinance data
aapl_8['Total Debt'] = aapl_debt.loc['2021-06-30']
aapl_8['Shares'] = aapl_shares.loc['2021-06-30']
aapl_8['Cash'] = aapl_cash.loc['2021-06-30']
aapl_8['EPS'] = 1.31
aapl_8['EBITDA'] = 26958000000

# View first five rows of data
aapl_8.head()

,Open,High,Low,Close,Adj Close,Volume,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,,,,,,
2021-07-06,140.070007,143.149994,140.070007,142.020004,140.375000,108181800,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-07,143.539993,144.889999,142.660004,144.570007,142.895493,104911600,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-08,141.580002,144.059998,140.669998,143.240005,141.580872,105575500,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-09,142.750000,145.649994,142.649994,145.110001,143.429230,99890800,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-12,146.210007,146.320007,144.000000,144.500000,142.826279,76299700,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000


## Part 3: Concat Data By Time Periods and Clean 

In [60]:
# Concat Data for all time periods 
aapl_df = aapl_8.append(aapl_7).append(aapl_6).append(aapl_5).append(aapl_4).append(aapl_3).append(aapl_2).append(aapl_1)
aapl_df.reset_index()
aapl_df.sort_values(by="Date", ascending = True)
aapl_df = aapl_df.drop(columns=["Open","High","Low","Adj Close", "Volume"])

# View first five rows of data
aapl_df.head()

,Close,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,
2021-07-06,142.020004,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-07,144.570007,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-08,143.240005,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-09,145.110001,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000
2021-07-12,144.500000,"121,791,000,000","16,556,942,000","19,001,000,000",1.31,26958000000


In [61]:
# Check dtypes 
aapl_df.dtypes

Close         float64
Total Debt     object
Shares         object
Cash           object
EPS           float64
EBITDA          int64
dtype: object

In [62]:
# Check how many rows
aapl_df.shape[0]

494

In [63]:
# Begin to clean strings dtypes by replacing commas 
aapl_df['Total Debt']= aapl_df['Total Debt'].str.replace(',','')
aapl_df['Shares']= aapl_df['Shares'].str.replace(',','')
aapl_df['Cash']= aapl_df['Cash'].str.replace(',','')

# View first five rows of data
aapl_df.head()

,Close,Total Debt,Shares,Cash,EPS,EBITDA
Date,,,,,,
2021-07-06,142.020004,121791000000,16556942000,19001000000,1.31,26958000000
2021-07-07,144.570007,121791000000,16556942000,19001000000,1.31,26958000000
2021-07-08,143.240005,121791000000,16556942000,19001000000,1.31,26958000000
2021-07-09,145.110001,121791000000,16556942000,19001000000,1.31,26958000000
2021-07-12,144.500000,121791000000,16556942000,19001000000,1.31,26958000000


In [64]:
# Change all columns to float64
aapl_df = aapl_df.astype(float)

# Check that all columns changed to float64
aapl_df.dtypes

Close         float64
Total Debt    float64
Shares        float64
Cash          float64
EPS           float64
EBITDA        float64
dtype: object

In [65]:
# Add fundamental ratios and enterprise value to each row of aapl_df
for i in aapl_df:
    aapl_df['P/E'] = aapl_df['Close'] / aapl_df['EPS']
for y in aapl_df:
    aapl_df['EV'] = (aapl_df['Shares'] * aapl_df['Close']) + aapl_df['Total Debt'] - aapl_df['Cash']
for z in aapl_df:
    aapl_df["EV/EBITDA"] = aapl_df['EV'] / aapl_df['EBITDA']

In [66]:
# View first 5 rows of data
aapl_df.head()

,Close,Total Debt,Shares,Cash,EPS,EBITDA,P/E,EV,EV/EBITDA
Date,,,,,,,,,
2021-07-06,142.020004,1.217910e+11,1.655694e+10,1.900100e+10,1.31,2.695800e+10,108.412217,2.454207e+12,91.038170
2021-07-07,144.570007,1.217910e+11,1.655694e+10,1.900100e+10,1.31,2.695800e+10,110.358784,2.496427e+12,92.604319
2021-07-08,143.240005,1.217910e+11,1.655694e+10,1.900100e+10,1.31,2.695800e+10,109.343516,2.474406e+12,91.787464
2021-07-09,145.110001,1.217910e+11,1.655694e+10,1.900100e+10,1.31,2.695800e+10,110.770993,2.505368e+12,92.935969
2021-07-12,144.500000,1.217910e+11,1.655694e+10,1.900100e+10,1.31,2.695800e+10,110.305344,2.495268e+12,92.561322


In [67]:
# Drop 'Total Debt', 'Shares', 'Cash', and 'EV' columns from dataset
aapl_df = aapl_df.drop(columns = ['Total Debt', 'Shares', 'Cash', 'EV'])

In [68]:
# View first and last 5 rows from data
aapl_df.head()

# Confirm total amount of rows
aapl_df.shape[0]

494

In [126]:
a_index = aapl_df.index

## Part 3.a: Find Daily Percent Change and Add To AAPL_DF

In [69]:
# Get and add 'Percent Change' to aapl_df
aapl_df['Percent Change'] = aapl_df['Close'].pct_change()

In [70]:
# Drop NaN values from dataset
aapl_df = aapl_df.dropna()

In [72]:
# View first and last 5 rows from data
display(aapl_df.head())
display(aapl_df.tail())

,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change
Date,,,,,,
2021-07-07,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955
2021-07-08,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200
2021-07-09,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055
2021-07-12,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204
2021-07-13,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889


,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change
Date,,,,,,
2023-06-26,185.270004,1.53,3.126000e+10,121.091506,95.874808,-0.007553
2023-06-27,188.059998,1.53,3.126000e+10,122.915031,97.278141,0.015059
2023-06-28,189.250000,1.53,3.126000e+10,123.692810,97.876698,0.006328
2023-06-29,189.589996,1.53,3.126000e+10,123.915030,98.047712,0.001797
2023-06-30,193.970001,1.53,3.126000e+10,126.777779,100.250802,0.023103


## Part 4: Read in Macro Data and Add To AAPL_DF

In [74]:
# Read .csv treasury data
treasury_df = pd.read_csv(
    Path("USTREASURY-YIELD.csv"),
    index_col = "Date"
)

In [75]:
# Display first and last 5 rows of data
display(treasury_df.head())
display(treasury_df.tail())

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
2023-06-30,5.24,5.39,5.43,5.47,5.40,4.87,4.49,4.13,3.97,3.81,4.06,3.85
2023-06-29,5.25,5.40,5.46,5.50,5.41,4.87,4.49,4.14,3.99,3.85,4.11,3.92
2023-06-28,5.17,5.32,5.44,5.47,5.32,4.71,4.32,3.97,3.83,3.71,4.00,3.81
2023-06-27,5.17,5.31,5.44,5.46,5.33,4.74,4.38,4.02,3.90,3.77,4.03,3.84
2023-06-26,5.17,5.31,5.50,5.45,5.27,4.65,4.30,3.96,3.85,3.72,4.01,3.83


,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04
1990-01-02,NaN,NaN,7.83,7.89,7.81,7.87,7.90,7.87,7.98,7.94,NaN,8.00


In [76]:
# Grab and only include data from "07/07/2021" to "06/30/2023"
treasury_df = treasury_df['2023-06-30' : '2021-07-07']

In [77]:
# Display first and last 5 rows of data
display(treasury_df.head())
display(treasury_df.tail())

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
2023-06-30,5.24,5.39,5.43,5.47,5.40,4.87,4.49,4.13,3.97,3.81,4.06,3.85
2023-06-29,5.25,5.40,5.46,5.50,5.41,4.87,4.49,4.14,3.99,3.85,4.11,3.92
2023-06-28,5.17,5.32,5.44,5.47,5.32,4.71,4.32,3.97,3.83,3.71,4.00,3.81
2023-06-27,5.17,5.31,5.44,5.46,5.33,4.74,4.38,4.02,3.90,3.77,4.03,3.84
2023-06-26,5.17,5.31,5.50,5.45,5.27,4.65,4.30,3.96,3.85,3.72,4.01,3.83


,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
2021-07-13,0.05,0.05,0.05,0.06,0.08,0.26,0.47,0.85,1.16,1.42,1.96,2.04
2021-07-12,0.05,0.05,0.05,0.06,0.08,0.23,0.43,0.81,1.13,1.38,1.93,2.00
2021-07-09,0.06,0.05,0.06,0.05,0.08,0.23,0.41,0.79,1.12,1.37,1.91,1.99
2021-07-08,0.06,0.05,0.06,0.06,0.07,0.19,0.37,0.74,1.06,1.30,1.84,1.91
2021-07-07,0.05,0.05,0.05,0.05,0.08,0.22,0.41,0.79,1.09,1.33,1.87,1.94


In [78]:
# Drop all columns besides '1 YR', '5 YR', and '30 YR'
treasury_df = treasury_df.drop(columns=['1 MO', '2 MO', '3 MO', '6 MO', '2 YR', '3 YR', '7 YR', '10 YR', '20 YR'])
trs_df = treasury_df

In [81]:
# Create new dataframe to be able to modify. Call this dataframe trs_df.
trs_df = treasury_df

# View first 5 rows of data
display(trs_df.head())
display(trs_df.tail())

,1 YR,5 YR,30 YR
Date,,,
2023-06-30,5.40,4.13,3.85
2023-06-29,5.41,4.14,3.92
2023-06-28,5.32,3.97,3.81
2023-06-27,5.33,4.02,3.84
2023-06-26,5.27,3.96,3.83


,1 YR,5 YR,30 YR
Date,,,
2021-07-13,0.08,0.85,2.04
2021-07-12,0.08,0.81,2.00
2021-07-09,0.08,0.79,1.99
2021-07-08,0.07,0.74,1.91
2021-07-07,0.08,0.79,1.94


In [130]:
# Drop the 'Datetime' index
trs_df = trs_df.reset_index(drop=True)

In [131]:
# Create NaN values on last 4 rows of trs_df
trs_df[-4:] = np.nan

In [132]:
# View last 5 rows in trs_df
trs_df.tail()

,1 YR,5 YR,30 YR
492,0.08,0.85,2.04
493,NaN,NaN,NaN
494,NaN,NaN,NaN
495,NaN,NaN,NaN
496,NaN,NaN,NaN


In [133]:
# Drop NaN values
trs_df = trs_df.dropna()

In [134]:
# Check last 5 rows of data
trs_df.tail()

,1 YR,5 YR,30 YR
488,0.07,0.70,1.81
489,0.08,0.79,1.93
490,0.07,0.78,1.92
491,0.08,0.80,1.98
492,0.08,0.85,2.04


## Part 4.a: Concat trs_df to aapl_df

In [136]:
# Create variable for aapl_df index
date = aapl_df.index
date

DatetimeIndex(['2021-07-07', '2021-07-08', '2021-07-09', '2021-07-12',
               '2021-07-13', '2021-07-14', '2021-07-15', '2021-07-16',
               '2021-07-19', '2021-07-20',
               ...
               '2023-06-16', '2023-06-20', '2023-06-21', '2023-06-22',
               '2023-06-23', '2023-06-26', '2023-06-27', '2023-06-28',
               '2023-06-29', '2023-06-30'],
              dtype='datetime64[ns]', name='Date', length=493, freq=None)

In [140]:
# Create results_df as a saved copy of aapl_df
results_df = aapl_df

In [141]:
# drop index from results_df
results_df = results_df.reset_index(drop=True)

In [142]:
# View first and last 5 rows of results_df
display(results_df.head())
display(results_df.tail())

,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change
0,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955
1,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200
2,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055
3,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204
4,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889


,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change
488,185.270004,1.53,3.126000e+10,121.091506,95.874808,-0.007553
489,188.059998,1.53,3.126000e+10,122.915031,97.278141,0.015059
490,189.250000,1.53,3.126000e+10,123.692810,97.876698,0.006328
491,189.589996,1.53,3.126000e+10,123.915030,98.047712,0.001797
492,193.970001,1.53,3.126000e+10,126.777779,100.250802,0.023103


In [144]:
# Append trs_df macro data to results_df
for i in trs_df:
    results_df['1 YR'] = trs_df['1 YR']
for j in trs_df:
    results_df['5 YR'] = trs_df['5 YR']
for k in trs_df:
    results_df['30 YR'] = trs_df['30 YR']

In [145]:
# View first 5 rows of data
results_df.head()

,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR
0,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85
1,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92
2,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81
3,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84
4,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83


In [146]:
# Reset index to 'Date' 
results_df['Date'] = pd.to_datetime(date)
results_df = results_df.set_index('Date')

# Make aapl_df = results_df
aapl_df = results_df

In [147]:
# Review first 5 rows of aapl_df
aapl_df.head()

,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR
Date,,,,,,,,,
2021-07-07,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85
2021-07-08,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92
2021-07-09,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81
2021-07-12,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84
2021-07-13,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83


## Part 5: Add Moving Averages to aapl_df

In [149]:
# Set Short (50) and Long Window (200)
short_window = 50
long_window = 200

In [150]:
# Isolate closing price
close_aapl = aapl_df.loc[:,['Close']]
close_aapl.head()

,Close
Date,
2021-07-07,144.570007
2021-07-08,143.240005
2021-07-09,145.110001
2021-07-12,144.500000
2021-07-13,145.639999


In [151]:
# Find 50 and 200 MA with rolling average
aapl_df['50 MA'] = close_aapl['Close'].rolling(window=short_window).mean()
aapl_df['200 MA'] = close_aapl['Close'].rolling(window=long_window).mean()

In [152]:
# Review first 5 rows of data
aapl_df.head()

,Close,EPS,EBITDA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR,50 MA,200 MA
Date,,,,,,,,,,,
2021-07-07,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85,NaN,NaN
2021-07-08,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92,NaN,NaN
2021-07-09,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81,NaN,NaN
2021-07-12,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84,NaN,NaN
2021-07-13,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83,NaN,NaN


In [153]:
# Replace NaN values with SimpleImputer. Fit and Transform this data.
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
aapl_df = imp_mean.fit_transform(aapl_df)

# Make aapl_df as a DataFrame and review first 5 rows in data
aapl_df = pd.DataFrame(aapl_df)
aapl_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85,155.018085,154.772202
1,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92,155.018085,154.772202
2,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81,155.018085,154.772202
3,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84,155.018085,154.772202
4,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83,155.018085,154.772202


In [154]:
# Add 'Date' back as index for aapl_df
aapl_df['Date'] = pd.to_datetime(date)
aapl_df = aapl_df.set_index('Date')

In [156]:
# Create columns for aapl_df
col = ['Close', 'EPS', 'EBIDTA', 'P/E', 'EV/EBITDA', 'Percent Change', '1 YR', '5 YR', '30 YR', '50 MA', '200 MA']

In [158]:
# Add columns back to aapl_df
aapl_df[col] = aapl_df 

In [161]:
# Add columns back to appl_df
aapl_df = aapl_df[col]

# Review first 5 rows of data
aapl_df.head()

,Close,EPS,EBIDTA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR,50 MA,200 MA
Date,,,,,,,,,,,
2021-07-07,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85,155.018085,154.772202
2021-07-08,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92,155.018085,154.772202
2021-07-09,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81,155.018085,154.772202
2021-07-12,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84,155.018085,154.772202
2021-07-13,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83,155.018085,154.772202


## Part 6: Create Buy or Sell Signal

In [162]:
# Create empty list and append a 'Buy' or 'Sell' signal 
rec = []
for i in aapl_df['Percent Change']:
    if i >= 0.0000000000000001:
        rec.append('Buy')
    else:
        rec.append('Sell')

In [163]:
# Create 'Recommendation column and assign 'rec' variable to it
aapl_df['Recommendation'] = rec

In [164]:
# Review first and last 5 rows of data
display(aapl_df.head())
display(aapl_df.tail())

,Close,EPS,EBIDTA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR,50 MA,200 MA,Recommendation
Date,,,,,,,,,,,,
2021-07-07,144.570007,1.31,2.695800e+10,110.358784,92.604319,0.017955,5.40,4.13,3.85,155.018085,154.772202,Buy
2021-07-08,143.240005,1.31,2.695800e+10,109.343516,91.787464,-0.009200,5.41,4.14,3.92,155.018085,154.772202,Sell
2021-07-09,145.110001,1.31,2.695800e+10,110.770993,92.935969,0.013055,5.32,3.97,3.81,155.018085,154.772202,Buy
2021-07-12,144.500000,1.31,2.695800e+10,110.305344,92.561322,-0.004204,5.33,4.02,3.84,155.018085,154.772202,Sell
2021-07-13,145.639999,1.31,2.695800e+10,111.175572,93.261482,0.007889,5.27,3.96,3.83,155.018085,154.772202,Buy


,Close,EPS,EBIDTA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR,50 MA,200 MA,Recommendation
Date,,,,,,,,,,,,
2023-06-26,185.270004,1.53,3.126000e+10,121.091506,95.874808,-0.007553,0.07,0.70,1.81,174.6788,154.07275,Sell
2023-06-27,188.059998,1.53,3.126000e+10,122.915031,97.278141,0.015059,0.08,0.79,1.93,175.1358,154.23400,Buy
2023-06-28,189.250000,1.53,3.126000e+10,123.692810,97.876698,0.006328,0.07,0.78,1.92,175.6162,154.40760,Buy
2023-06-29,189.589996,1.53,3.126000e+10,123.915030,98.047712,0.001797,0.08,0.80,1.98,176.0786,154.57575,Buy
2023-06-30,193.970001,1.53,3.126000e+10,126.777779,100.250802,0.023103,0.08,0.85,2.04,176.6054,154.77330,Buy


In [165]:
# Sort Data By Date: 
aapl_df = aapl_df.sort_values(by='Date' ,ascending=False)
pd.to_datetime(aapl_df.index).strftime('%Y-%m-%d')

Index(['2023-06-30', '2023-06-29', '2023-06-28', '2023-06-27', '2023-06-26',
       '2023-06-23', '2023-06-22', '2023-06-21', '2023-06-20', '2023-06-16',
       ...
       '2021-07-20', '2021-07-19', '2021-07-16', '2021-07-15', '2021-07-14',
       '2021-07-13', '2021-07-12', '2021-07-09', '2021-07-08', '2021-07-07'],
      dtype='object', name='Date', length=493)

In [166]:
# Review first 5 rows of data
aapl_df.head()

,Close,EPS,EBIDTA,P/E,EV/EBITDA,Percent Change,1 YR,5 YR,30 YR,50 MA,200 MA,Recommendation
Date,,,,,,,,,,,,
2023-06-30,193.970001,1.53,3.126000e+10,126.777779,100.250802,0.023103,0.08,0.85,2.04,176.6054,154.77330,Buy
2023-06-29,189.589996,1.53,3.126000e+10,123.915030,98.047712,0.001797,0.08,0.80,1.98,176.0786,154.57575,Buy
2023-06-28,189.250000,1.53,3.126000e+10,123.692810,97.876698,0.006328,0.07,0.78,1.92,175.6162,154.40760,Buy
2023-06-27,188.059998,1.53,3.126000e+10,122.915031,97.278141,0.015059,0.08,0.79,1.93,175.1358,154.23400,Buy
2023-06-26,185.270004,1.53,3.126000e+10,121.091506,95.874808,-0.007553,0.07,0.70,1.81,174.6788,154.07275,Sell


In [167]:
# Reorganize columns 
new_col = ['Close', 'Percent Change', '50 MA', '200 MA', 'EPS', 'EBIDTA', 'P/E', 'EV/EBITDA', '1 YR', '5 YR', '30 YR', 'Recommendation']

In [168]:
# Reorganize columns to aapl_df
aapl_df = aapl_df[new_col]

In [169]:
# Review first 5 rows of data
aapl_df.head()

,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation
Date,,,,,,,,,,,,
2023-06-30,193.970001,0.023103,176.6054,154.77330,1.53,3.126000e+10,126.777779,100.250802,0.08,0.85,2.04,Buy
2023-06-29,189.589996,0.001797,176.0786,154.57575,1.53,3.126000e+10,123.915030,98.047712,0.08,0.80,1.98,Buy
2023-06-28,189.250000,0.006328,175.6162,154.40760,1.53,3.126000e+10,123.692810,97.876698,0.07,0.78,1.92,Buy
2023-06-27,188.059998,0.015059,175.1358,154.23400,1.53,3.126000e+10,122.915031,97.278141,0.08,0.79,1.93,Buy
2023-06-26,185.270004,-0.007553,174.6788,154.07275,1.53,3.126000e+10,121.091506,95.874808,0.07,0.70,1.81,Sell


## Part 7: Sentiment Analysis

In [170]:
# Read in Sentiment Analysis 
sentiment_analysis_df = pd.read_csv(
    Path("sentiment_analysis.csv"),
    index_col = "Date")

In [171]:
# Sort sentiment values by 'Date'
sentiment_analysis_df = sentiment_analysis_df.sort_values(by='Date' ,ascending=False)

In [172]:
# Review first 5 rows
sentiment_analysis_df.head()

,label,score
Date,,
2023-06-30,POSITIVE,0.748121
2023-06-29,POSITIVE,0.748121
2023-06-28,POSITIVE,0.999122
2023-06-27,POSITIVE,0.977651
2023-06-26,POSITIVE,0.691947


In [173]:
# Set date_sa as index variable name
date_sa = sentiment_analysis_df.index

In [174]:
# Create sa2 as a 'copy' version of sentiment_analysis_df
# Drop index
sa2 = sentiment_analysis_df.reset_index(drop=True)

In [175]:
# Review first 5 rows
sa2.head()

,label,score
0,POSITIVE,0.748121
1,POSITIVE,0.748121
2,POSITIVE,0.999122
3,POSITIVE,0.977651
4,POSITIVE,0.691947


In [176]:
# Remove columns that are not required
sa2[-1306:] = np.nan

In [177]:
# Review last 5 rows of data
sa2.tail()

,label,score
1794,NaN,NaN
1795,NaN,NaN
1796,NaN,NaN
1797,NaN,NaN
1798,NaN,NaN


In [178]:
# Drop NaN values of sa2
sa2 = sa2.dropna()

In [179]:
# Review last 5 rows of data
sa2.tail()

,label,score
488,POSITIVE,0.985896
489,NEGATIVE,0.892429
490,NEGATIVE,0.998747
491,POSITIVE,0.947093
492,POSITIVE,0.593836


In [181]:
# Remove index from aap_df and create a copy version of this as 'aapl_no_dt'
aapl_no_dt = aapl_df.reset_index(drop=True)

# Review first and last 5 rows of data
display(aapl_no_dt.head())
display(aapl_no_dt.tail())

,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation
0,193.970001,0.023103,176.6054,154.77330,1.53,3.126000e+10,126.777779,100.250802,0.08,0.85,2.04,Buy
1,189.589996,0.001797,176.0786,154.57575,1.53,3.126000e+10,123.915030,98.047712,0.08,0.80,1.98,Buy
2,189.250000,0.006328,175.6162,154.40760,1.53,3.126000e+10,123.692810,97.876698,0.07,0.78,1.92,Buy
3,188.059998,0.015059,175.1358,154.23400,1.53,3.126000e+10,122.915031,97.278141,0.08,0.79,1.93,Buy
4,185.270004,-0.007553,174.6788,154.07275,1.53,3.126000e+10,121.091506,95.874808,0.07,0.70,1.81,Sell


,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation
488,145.639999,0.007889,155.018085,154.772202,1.31,2.695800e+10,111.175572,93.261482,5.27,3.96,3.83,Buy
489,144.500000,-0.004204,155.018085,154.772202,1.31,2.695800e+10,110.305344,92.561322,5.33,4.02,3.84,Sell
490,145.110001,0.013055,155.018085,154.772202,1.31,2.695800e+10,110.770993,92.935969,5.32,3.97,3.81,Buy
491,143.240005,-0.009200,155.018085,154.772202,1.31,2.695800e+10,109.343516,91.787464,5.41,4.14,3.92,Sell
492,144.570007,0.017955,155.018085,154.772202,1.31,2.695800e+10,110.358784,92.604319,5.40,4.13,3.85,Buy


In [182]:
# Add 'News' and 'News Score' to aapl_no_dt
for i in aapl_no_dt:
    aapl_no_dt['News'] = sa2['label']
    
for j in aapl_no_dt:
    aapl_no_dt['News Score'] = sa2['score']

In [183]:
# Review first 5 rows of data
aapl_no_dt.head()

,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation,News,News Score
0,193.970001,0.023103,176.6054,154.77330,1.53,3.126000e+10,126.777779,100.250802,0.08,0.85,2.04,Buy,POSITIVE,0.748121
1,189.589996,0.001797,176.0786,154.57575,1.53,3.126000e+10,123.915030,98.047712,0.08,0.80,1.98,Buy,POSITIVE,0.748121
2,189.250000,0.006328,175.6162,154.40760,1.53,3.126000e+10,123.692810,97.876698,0.07,0.78,1.92,Buy,POSITIVE,0.999122
3,188.059998,0.015059,175.1358,154.23400,1.53,3.126000e+10,122.915031,97.278141,0.08,0.79,1.93,Buy,POSITIVE,0.977651
4,185.270004,-0.007553,174.6788,154.07275,1.53,3.126000e+10,121.091506,95.874808,0.07,0.70,1.81,Sell,POSITIVE,0.691947


In [185]:
# Add date index back to aapl_no_dt
aapl_no_dt['Date'] = pd.to_datetime(date)
aapl_no_dt = aapl_no_dt.set_index('Date')

In [187]:
# Review first 5 rows of data
aapl_no_dt.head()

,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation,News,News Score
Date,,,,,,,,,,,,,,
2021-07-07,193.970001,0.023103,176.6054,154.77330,1.53,3.126000e+10,126.777779,100.250802,0.08,0.85,2.04,Buy,POSITIVE,0.748121
2021-07-08,189.589996,0.001797,176.0786,154.57575,1.53,3.126000e+10,123.915030,98.047712,0.08,0.80,1.98,Buy,POSITIVE,0.748121
2021-07-09,189.250000,0.006328,175.6162,154.40760,1.53,3.126000e+10,123.692810,97.876698,0.07,0.78,1.92,Buy,POSITIVE,0.999122
2021-07-12,188.059998,0.015059,175.1358,154.23400,1.53,3.126000e+10,122.915031,97.278141,0.08,0.79,1.93,Buy,POSITIVE,0.977651
2021-07-13,185.270004,-0.007553,174.6788,154.07275,1.53,3.126000e+10,121.091506,95.874808,0.07,0.70,1.81,Sell,POSITIVE,0.691947


In [188]:
# Assign aapl_df to aapl_no_dt
aapl_df = aapl_no_dt

# Review first 5 rows of data
aapl_df.head()

,Close,Percent Change,50 MA,200 MA,EPS,EBIDTA,P/E,EV/EBITDA,1 YR,5 YR,30 YR,Recommendation,News,News Score
Date,,,,,,,,,,,,,,
2021-07-07,193.970001,0.023103,176.6054,154.77330,1.53,3.126000e+10,126.777779,100.250802,0.08,0.85,2.04,Buy,POSITIVE,0.748121
2021-07-08,189.589996,0.001797,176.0786,154.57575,1.53,3.126000e+10,123.915030,98.047712,0.08,0.80,1.98,Buy,POSITIVE,0.748121
2021-07-09,189.250000,0.006328,175.6162,154.40760,1.53,3.126000e+10,123.692810,97.876698,0.07,0.78,1.92,Buy,POSITIVE,0.999122
2021-07-12,188.059998,0.015059,175.1358,154.23400,1.53,3.126000e+10,122.915031,97.278141,0.08,0.79,1.93,Buy,POSITIVE,0.977651
2021-07-13,185.270004,-0.007553,174.6788,154.07275,1.53,3.126000e+10,121.091506,95.874808,0.07,0.70,1.81,Sell,POSITIVE,0.691947


## Part 8: Export as a .csv file

In [189]:
## Export to .csv
aapl_df.to_csv('analyst_rating.csv')